In [1]:
# /// script
# requires-python = ">=3.9"
# dependencies = [
#     "geoarrow-rust-io>=0.4.0b2",
#     "lonboard",
#     "requests",
# ]
# ///

## Reading spatially-partitioned GeoParquet

This notebook is an example of reading spatially-partitioned GeoParquet directly from the cloud, with no server in between. This will load a spatial filter from Overture's buildings dataset on the fly, even though the dataset has over 2 billion rows.

This notebook uses [`juv`](https://github.com/manzt/juv) for dependency management. First [install `uv`](https://docs.astral.sh/uv/) then launch this notebook with:

```
uvx juv run overture_buildings.ipynb
```

### Imports

In [2]:
import lonboard
import requests

from IPython.display import JSON

from geoarrow.rust.io import ParquetDataset
from geoarrow.rust.io.store import S3Store

Overture publishes a JSON manifest with the Parquet files in each release. We'll fetch this and inspect it:

In [14]:
manifest_url = "https://raw.githubusercontent.com/OvertureMaps/io-site/refs/heads/manifest_driven_downloads/site/src/09-18-manifest.json"
manifest = requests.get(manifest_url).json()

In [16]:
JSON?

Init signature:
JSON(
    data=None,
    url=None,
    filename=None,
    expanded=False,
    metadata=None,
    root='root',
    **kwargs,
)
Docstring:     
JSON expects a JSON-able dict or list

not an already-serialized JSON string.

Scalar types (None, number, string) are not allowed, only dict or list containers.
Init docstring:
Create a JSON display object given raw data.

Parameters
----------
data : dict or list
    JSON data to display. Not an already-serialized JSON string.
    Scalar types (None, number, string) are not allowed, only dict
    or list containers.
url : unicode
    A URL to download the data from.
filename : unicode
    Path to a local file to load the data from.
expanded : boolean
    Metadata to control whether a JSON display component is expanded.
metadata : dict
    Specify extra metadata to attach to the json display object.
root : str
    The name of the root element of the JSON tree
File:           ~/Library/Caches/uv/archive-v0/5cqfHDkg-hB_shnDMgoqh/li

In [18]:
JSON(manifest)

<IPython.core.display.JSON object>

Next we'll construct the Parquet URLs for each of the building files:

In [19]:
s3_url = "s3://"

s3_url += manifest["s3_location"]
buildings_manifest = manifest["themes"][2]

s3_url += buildings_manifest["relative_path"]
buildings_type = buildings_manifest["types"][0]

s3_url += buildings_type["relative_path"]

file_names = [s3_url + "/" + file["name"] for file in buildings_type["files"]]

In [20]:
file_names[:5]

['s3://overturemaps-us-west-2/release/2024-09-18.0/theme=buildings/type=building/part-00000-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 's3://overturemaps-us-west-2/release/2024-09-18.0/theme=buildings/type=building/part-00001-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 's3://overturemaps-us-west-2/release/2024-09-18.0/theme=buildings/type=building/part-00002-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 's3://overturemaps-us-west-2/release/2024-09-18.0/theme=buildings/type=building/part-00003-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 's3://overturemaps-us-west-2/release/2024-09-18.0/theme=buildings/type=building/part-00004-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet']

Now we'll construct an `S3Store` to tell the GeoParquet reader how to access S3.

This class presents the _same API_ as classes in [`obstore.store`](https://developmentseed.org/obstore/latest/api/store/aws/), but since we're using this with `geoarrow.rust.io`, we need to ensure we import the `S3Store` class from `geoarrow.rust.io.store`, not `obstore.store`.

In [21]:
store = S3Store.from_url("s3://overturemaps-us-west-2", config={"SKIP_SIGNATURE": "True", "REGION": "us-west-2"})

Remove the `s3://overturemaps-us-west-2/` prefix from each path:

In [25]:
paths = [name[28:] for name in file_names]

In [26]:
paths[:5]

['release/2024-09-18.0/theme=buildings/type=building/part-00000-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 'release/2024-09-18.0/theme=buildings/type=building/part-00001-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 'release/2024-09-18.0/theme=buildings/type=building/part-00002-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 'release/2024-09-18.0/theme=buildings/type=building/part-00003-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet',
 'release/2024-09-18.0/theme=buildings/type=building/part-00004-d0628531-2fbd-47aa-8608-f61ff708b14c-c000.zstd.parquet']

There are 208 different GeoParquet files in the directory:

In [28]:
len(paths)

208

Next, we'll create the `ParquetDataset` class. This represents many GeoParquet files of the same schema across a directory.

When you create this class, it fetches the metadata from all 208 GeoParquet files, which takes a few seconds:

In [29]:
%%time
dataset = ParquetDataset(paths, store)

CPU times: user 944 ms, sys: 792 ms, total: 1.74 s
Wall time: 3.72 s


Now we can see some information about the datasets, such as its row group information, number of rows, and schema:

In [31]:
dataset.num_row_groups

14885

In [32]:
dataset.num_rows

2375911049

In [33]:
dataset.schema_arrow

arro3.core.Schema
------------
id: Utf8
geometry: List(Field { name: "polygons", data_type: List(Field { name: "rings", data_type: List(Field { name: "vertices", data_type: FixedSizeList(Field { name: "xy", data_type: Float64, nullable: false, dict_id: 0, dict_is_ordered: false, metadata: {} }, 2), nullable: false, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: false, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: false, dict_id: 0, dict_is_ordered: false, metadata: {} })
bbox: Struct([Field { name: "xmin", data_type: Float32, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "xmax", data_type: Float32, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "ymin", data_type: Float32, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "ymax", data_type: Float32, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }])
version: Int32
sources: List(Field {

Now we can fetch data from a specific bounding box. Note that the bounding box needs to have the same coordinate system as the data

In [36]:
%%time
nyc_buildings = dataset.read(bbox=(-74.021242, 40.697751, -73.947232, 40.790611))

CPU times: user 541 ms, sys: 740 ms, total: 1.28 s
Wall time: 12.7 s


In [37]:
len(nyc_buildings)

47650

In [38]:
lonboard.viz(nyc_buildings)

/Users/kyle/Library/Caches/uv/archive-v0/AmyN-6PWaSEhdrWl-TudB/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…